In [1]:
import pandas as pd
import glob
import csv
import warnings
import datetime as dt
import numpy as np

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.set_option('display.max_columns', 500)

In [2]:
obras= glob.glob(r"C:\Users\thiago.mussolini\Desktop\OBRAS\py-obras\*.xlsx") 
obras 

['C:\\Users\\thiago.mussolini\\Desktop\\OBRAS\\py-obras\\L30_PROG-SEMANAL-OBRAS-24-01-22-30-01-22.xlsx']

In [4]:
cabecalho = ['LOTE', 'RODOVIA', 'CÓD. DO SERVIÇO', 'DETALHAMENTO DO SERVIÇO',
       'KM INICIAL', 'LAT KM INICIAL', 'LON KM INICIAL', 'KM FINAL',
       'LAT KM FINAL', 'LON KM FINAL', 'SENTIDO', 'DATA INÍCIO', 'HORA INÍCIO',
       'DATA FIM', 'HORA FIM', 'FI1', 'FI2', 'FI3', 'FI4', 'FI5', 'FI6',
       'FIAC', 'ALT.', 'EMPRESA RESPONSÁVEL',
       'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA',
       'ART DO RESPONSÁVEL TÉCNICO', 'NÍVEL DE SERVIÇO C/ INTERVENÇÃO',
       'APOIO PMRV', 'MUNICÍPIO', 'OBSERVAÇÕES GERAIS']

In [ ]:
df_lista=[]
for obra in obras:
    print("planilha: ", obra)
    extensao = obra.split('.')[-1]
    print("extensao: ", extensao)
    df= pd.read_excel(
        obra, sheet_name="DADOS", 
        dtype={'DATA INÍCIO': object, 'DATA FIM': object, 'OBSERVAÇÕES GERAIS': object, 'DETALHAMENTO DO SERVIÇO': object, 'EMPRESA RESPONSÁVEL': object,
        'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA': object, 'APOIO PMRV': object, 'MUNICÍPIO': object, 'ART DO RESPONSÁVEL TÉCNICO': object},
        names=cabecalho, usecols=[x for x in range(30)]
    )
    df = df[~(df.LOTE.isnull())]
    data2 = pd.to_datetime(df['DATA FIM'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
    df['DATA FIM'] = pd.to_datetime(df['DATA FIM'], errors='coerce', format='%d/%m/%Y')
    df['DATA FIM'] = df['DATA FIM'].fillna(data2)
    data2 = pd.to_datetime(df['DATA INÍCIO'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
    df['DATA INÍCIO'] = pd.to_datetime(df['DATA INÍCIO'], errors='coerce', format='%d/%m/%Y')
    df['DATA INÍCIO'] = df['DATA INÍCIO'].fillna(data2)
    df['KM INICIAL'] = df['KM INICIAL'].apply(lambda valor: str(valor).replace('+', ''))
    df['KM FINAL'] = df['KM FINAL'].apply(lambda valor: str(valor).replace('+', ''))
    df['KM INICIAL'] = df['KM INICIAL'].apply(lambda valor: str(valor).replace(',', ''))
    df['KM FINAL'] = df['KM FINAL'].apply(lambda valor: str(valor).replace(',', ''))
    df['KM INICIAL'] = pd.to_numeric(df['KM INICIAL'])
    df['KM FINAL'] = pd.to_numeric(df['KM FINAL'])
    df['KM INICIAL'] = df['KM INICIAL'] / 1000
    df['KM FINAL'] = df['KM FINAL'] / 1000
    df['OBSERVAÇÕES GERAIS'] = df['OBSERVAÇÕES GERAIS'].replace(r'\s+|\\n', ' ', regex=True)
    df['DETALHAMENTO DO SERVIÇO'] = df['DETALHAMENTO DO SERVIÇO'].replace(r'\s+|\\n', ' ', regex=True)
    df['EMPRESA RESPONSÁVEL'] = df['EMPRESA RESPONSÁVEL'].replace(r'\s+|\\n', ' ', regex=True)
    df['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'] = df['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'].replace(r'\s+|\\n', ' ', regex=True)
    df['APOIO PMRV'] = df['APOIO PMRV'].replace(r'\s+|\\n', ' ', regex=True)
    df['MUNICÍPIO'] = df['MUNICÍPIO'].replace(r'\s+|\\n', ' ', regex=True)
    df = df.iloc[:, :30]
    df_rodovia_nulo = df['RODOVIA'].isnull()
    df= df[~df_rodovia_nulo]
    df_lista.append(df)

In [6]:
df_obras = pd.concat(df_lista, axis=0, ignore_index=True)

In [7]:
df_obras.reset_index(drop=True, inplace=True)

In [8]:
df_obras["LOTE"] = df_obras["LOTE"].astype(int)

In [9]:
df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('OL','LO'))
df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('IE','EI'))
df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('SN','NS'))
df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Leste','L'))
df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Norte','N'))
df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Oeste','O'))
df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Sul','S'))
df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Leste / Oeste','LO'))
df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Norte / Sul','NS'))
df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].apply(lambda valor: str(valor).replace('-SP',''))
df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].apply(lambda valor: str(valor).replace(' -SP',''))
df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].apply(lambda valor: str(valor).replace(' - SP',''))
df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].apply(lambda valor: valor.split('/')[0])
df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'] = df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'].replace(np.nan,'NÃO INFORMADO')
df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].str.upper()
df_obras['OBSERVAÇÕES GERAIS'] = df_obras['OBSERVAÇÕES GERAIS'].str.upper()
df_obras['RODOVIA'] = df_obras['RODOVIA'].str.upper()
df_obras['DETALHAMENTO DO SERVIÇO'] = df_obras['DETALHAMENTO DO SERVIÇO'].str.upper()
df_obras['APOIO PMRV'] = df_obras['APOIO PMRV'].str.upper()
df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'] = df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'].str.upper()
df_obras['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'] = df_obras['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'].str.upper()
df_obras['EMPRESA RESPONSÁVEL'] = df_obras['EMPRESA RESPONSÁVEL'].str.upper()
df_obras['SENTIDO'] = df_obras['SENTIDO'].str.upper()
df_obras['FI1'] = df_obras['FI1'].str.upper()
df_obras['FI2'] = df_obras['FI2'].str.upper()
df_obras['FI3'] = df_obras['FI3'].str.upper()
df_obras['FI4'] = df_obras['FI4'].str.upper()
df_obras['FI5'] = df_obras['FI5'].str.upper()
df_obras['FI6'] = df_obras['FI6'].str.upper()
df_obras['FIAC'] = df_obras['FIAC'].str.upper()
df_obras['FI1'] = df_obras['FI1'].apply(lambda valor: str(valor).replace('S','1'))
df_obras['FI2'] = df_obras['FI2'].apply(lambda valor: str(valor).replace('S','2'))
df_obras['FI3'] = df_obras['FI3'].apply(lambda valor: str(valor).replace('S','3'))
df_obras['FI4'] = df_obras['FI4'].apply(lambda valor: str(valor).replace('S','4'))
df_obras['FI5'] = df_obras['FI5'].apply(lambda valor: str(valor).replace('S','5'))
df_obras['FI6'] = df_obras['FI6'].apply(lambda valor: str(valor).replace('S','6'))
df_obras['FIAC'] = df_obras['FIAC'].apply(lambda valor: str(valor).replace('S','ACOS'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FI1'] + ',' + df_obras['FI2'] + ',' + df_obras['FI3'] + ','+ df_obras['FI4'] + ',' + df_obras['FI5'] + ',' + df_obras['FI6'] + ',' + df_obras['FIAC']

In [11]:
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('N','NENHUMA'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('6NENHUMA','6'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('4NENHUMA','4'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('3NENHUMA','3'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('2NENHUMA','2'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('1NENHUMA','1'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('NENHUMA,',''))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('1,NENHUMA','1'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('2,NENHUMA','2'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('3,NENHUMA','3'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('4,NENHUMA','4'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('5,NENHUMA','5'))
df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('6,NENHUMA','6'))



In [12]:
df_obras = df_obras[['LOTE', 'RODOVIA', 'CÓD. DO SERVIÇO', 'DETALHAMENTO DO SERVIÇO',
       'KM INICIAL', 'LAT KM INICIAL', 'LON KM INICIAL', 'KM FINAL',
       'LAT KM FINAL', 'LON KM FINAL', 'SENTIDO', 'DATA INÍCIO', 'HORA INÍCIO',
       'DATA FIM', 'HORA FIM', 'FI1', 'FI2', 'FI3', 'FI4', 'FI5', 'FI6',
       'FIAC', 'ALT.', 'FAIXAS INTERDITADAS' , 'EMPRESA RESPONSÁVEL',
       'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA',
       'ART DO RESPONSÁVEL TÉCNICO', 'NÍVEL DE SERVIÇO C/ INTERVENÇÃO',
       'APOIO PMRV', 'MUNICÍPIO', 'OBSERVAÇÕES GERAIS']]

In [14]:
df_obras.to_csv("OBRAS_24_01_2022_a_30_01_2022.csv", encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC, index_label="ID")

In [16]:
csv_semana_passada = pd.read_csv(r'C:\Users\thiago.mussolini\Desktop\OBRAS\py-obras\OBRAS_17_01_2022_a_23_01_2022.csv', sep=',', low_memory=False, dtype= {'LOTE': object, 'CÓD. DO SERVIÇO': object})
csv_semana_atual = pd.read_csv(r'C:\Users\thiago.mussolini\Desktop\OBRAS\py-obras\OBRAS_24_01_2022_a_30_01_2022.csv', sep=',', low_memory=False, dtype= {'LOTE': object, 'CÓD. DO SERVIÇO': object})

In [17]:
obras_consolidada = pd.concat([csv_semana_passada, csv_semana_atual])

In [18]:
obras_consolidada = obras_consolidada[['LOTE', 'RODOVIA', 'CÓD. DO SERVIÇO',
       'DETALHAMENTO DO SERVIÇO', 'KM INICIAL', 'LAT KM INICIAL',
       'LON KM INICIAL', 'KM FINAL', 'LAT KM FINAL', 'LON KM FINAL', 'SENTIDO',
       'DATA INÍCIO', 'HORA INÍCIO', 'DATA FIM', 'HORA FIM', 'FI1', 'FI2',
       'FI3', 'FI4', 'FI5', 'FI6', 'FIAC', 'ALT.', 'FAIXAS INTERDITADAS',
       'EMPRESA RESPONSÁVEL', 'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA',
       'ART DO RESPONSÁVEL TÉCNICO', 'NÍVEL DE SERVIÇO C/ INTERVENÇÃO',
       'APOIO PMRV', 'MUNICÍPIO', 'OBSERVAÇÕES GERAIS']]

In [19]:
obras_consolidada.to_csv("OBRAS.CSV", sep=',', index=False)